# This Notebook is used for generating the data for Meal planning for the new Millennium (MnM) problem

## 1. Import all the necessary library

In [13]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import multiprocessing
import pandas as pd 
import numpy as np

!pip install html5lib

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


## 2. Define EP_Recipe class to store all the data.

In [2]:
class EP_Recipe():
    title = None
    rating = None
    personal_rating = []
    calories = None
    sodium = None
    fat = None
    protein = None

    def get_title(self, page):
        return page.find('h1', {'itemprop': 'name'}).text

    def get_rating(self, page):
        try:
            return float(page.find_all('span', {'class': 'rating'})[-1].text.split('/')[0]) 
        except:
            return None

    def build_recipie(self, page):
        #super(EP_Recipe, self).build_recipie(page)
        self.title = self.get_title(page)
        self.rating = self.get_rating(page)
        self.calories = self.get_calories(page)
        self.sodium = self.get_sodium(page)
        self.fat = self.get_fat(page)
        self.protein = self.get_protein(page)
        self.personal_rating = self.get_personal_rating(page)

    def get_calories(self,page):
        try:
            return float(page.find('span',{'class':'nutri-data','itemprop':'calories'}).text)
        except:
            return None

    def get_sodium(self,page):
        try:
            return float(page.find('span',{'class':'nutri-data','itemprop':'sodiumContent'}).text.split(' ')[0])
        except:
            return None

    def get_fat(self, page):
        try:
            return float(page.find('span', {'class': 'nutri-data', 'itemprop': 'fatContent'}).text.split(' ')[0])
        except:
            return None

    def get_protein(self, page):
        try:
            return float(page.find('span', {'class': 'nutri-data', 'itemprop': 'proteinContent'}).text.split(' ')[0])
        except:
            return None
    
    def get_personal_rating(self, page):
        try: 
            p_ratings = page.findAll('img', {'class': 'fork-rating'})
            p_rating = []
            for i in p_ratings:
                p_rating.append(int(i['src'].split('/')[-1].split('_')[0]))
            return p_rating
        except:
            return None


    def __init__(self, page):
        print('attempting to build from: '+page)
        try:
            self.build_recipie(bs(url(page), 'html.parser'))
        except Exception as x:
            print('Could not build from %s, %s'%(page,x))

## 3. Find all the addresses for recipes and store them in ep_urls

In [10]:
##choose a bunch of menus to find the recipes
all_url = ['https://www.epicurious.com/recipes-menus/what-to-cook-this-weekend-february-22-24-gallery',
          'https://www.epicurious.com/recipes-menus/what-to-cook-this-weekend-february-8-10-gallery',
           "https://www.epicurious.com/ingredients/acorn-delicata-kabocha-spaghetti-squash-winter-recipes-gallery",
           'https://www.epicurious.com/recipes-menus/easy-dinner-recipes-for-cook90-gallery',
          'https://www.epicurious.com/recipes-menus/our-favorite-cook90-lunches-gallery',
          'https://www.epicurious.com/recipes-menus/make-ahead-weeknight-dinners-stew-soup-freezer-casserole-quick-easy-recipes-gallery']


ep_urls = set()  ##Store the web address of recipes
for i in all_url:        
    initializer = url(i)
    res = bs(initializer.read(),"html5lib")
    for div in res.findAll('div', {'class': 'gallery-slide-caption__dek-container'}):
            ep_urls.update([div.find('a')['href']]) 

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

## 4. Scrape the website of recipes and generate the data. Store the data as recipes_data.json.

In [9]:
p = multiprocessing.Pool(4)
output = p.map(EP_Recipe,ep_urls)
pickle.dump(output,open('epi_recipes.final','wb'))

data = pickle.load(open('epi_recipes.final','rb'))
ar = []
for i in data:
    ar.append(i.__dict__)
pickle.dump(ar,open('epi_recipe_dict_form.dict','wb'))

with open('recipes_data.json', 'w') as fp:
    json.dump(ar, fp)

In [17]:
df = pd.read_json('recipes_data.json')
df

,calories,fat,personal_rating,protein,rating,sodium,title
0,488.0,32.0,"[4, 4, 4, 4, 4, 1, 4, 4, 3]",29.0,3.5,985.0,Freeform Chicken Meatballs with Carrots and Yo...
1,NaN,NaN,"[3, 4]",NaN,3.5,NaN,Jammy Eggs and Feta Flatbreads with Herbs
2,289.0,19.0,"[3, 2, 4, 4, 4, 4, 4, 3]",7.0,3.5,660.0,Miso-Harissa Delicata Squash and Brussels Spro...
3,572.0,10.0,"[2, 4, 3, 2, 4, 4, 3, 1, 4, 4, 3, 4, 4, 4, 4, ...",53.0,3.0,1592.0,Slow-Cooker White Chicken Chili
4,645.0,45.0,"[4, 4]",23.0,4.0,670.0,Green Goddess Tuna Salad Sandwich
5,194.0,11.0,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",2.0,4.0,110.0,Orange Sweet Rolls
6,NaN,NaN,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, ...",NaN,4.0,NaN,Skillet Chicken Pot Pie with Butternut Squash
7,156.0,4.0,"[3, 4, 4, 3]",2.0,3.5,631.0,Winter Squash Agrodolce
8,3597.0,223.0,"[4, 4, 3, 1, 2, 0, 4, 0, 0, 4, 4, 0, 4, 4, 3, ...",33.0,3.5,1869.0,Pumpkin Icebox Pie With Snickerdoodle Crust
9,708.0,41.0,"[1, 2, 3, 4, 2, 1, 4, 2, 3, 3, 1, 3, 1, 3, 4, 2]",20.0,2.5,733.0,Veggie Burgers with Zucchini and Corn


In [18]:
df = df.dropna(axis=0) #delete row with NaN 

In [19]:
df = df[df['personal_rating'].map(len) > 13]

In [41]:
df

,calories,fat,personal_rating,protein,rating,sodium,title
3,572.0,10.0,"[2, 4, 3, 2, 4, 4, 3, 1, 4, 4, 3, 4, 4, 4, 4, ...",53.0,3.0,1592.0,Slow-Cooker White Chicken Chili
8,3597.0,223.0,"[4, 4, 3, 1, 2, 4, 4, 4, 0, 4, 4, 3, 3, 2, 4, ...",33.0,3.5,1869.0,Pumpkin Icebox Pie With Snickerdoodle Crust
9,708.0,41.0,"[1, 2, 3, 4, 2, 1, 4, 2, 3, 3, 1, 3, 1, 3, 4, 2]",20.0,2.5,733.0,Veggie Burgers with Zucchini and Corn
11,263.0,18.0,"[3, 1, 4, 2, 4, 2, 4, 3, 4, 1, 1, 4, 4]",4.0,3.0,50.0,"Dandelion Salad with Pomegranate Seeds, Pine N..."
12,346.0,19.0,"[4, 4, 3, 4, 4, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, ...",3.0,4.0,220.0,Pumpkin Whoopie Pies
15,257.0,21.0,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4]",10.0,4.0,659.0,Kabocha Squash and Pork Stir-Fry
20,534.0,14.0,"[4, 4, 4, 4, 4, 4, 1, 4, 4, 3, 2, 4, 4, 4, 3, ...",45.0,4.0,968.0,White Chicken Chili
23,581.0,36.0,"[4, 4, 4, 3, 4, 4, 4, 4, 4, 1, 3, 4, 3, 4, 4, ...",36.0,3.5,1044.0,"Our Favorite Lasagna with Sausage, Spinach, an..."
28,606.0,15.0,"[3, 2, 3, 4, 3, 4, 2, 4, 3, 2, 3, 3, 4, 4, 3, ...",27.0,3.5,1365.0,One-Pot Curried Cauliflower with Couscous and ...
29,1062.0,60.0,"[4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 3, 4, ...",48.0,3.5,2000.0,Our Favorite Spaghetti and Meatballs
